In [174]:

from scipy.stats.qmc import LatinHypercube as LHSampler
from scipy.stats import qmc 
from scipy.integrate import quad, dblquad, tplquad
from matplotlib import pyplot as plt
import numpy as np
from src.utils import plotLHS, concat, H
from src.eLHS_old import *
import json
from math import floor, ceil
from src.fancyplotter import usePlotSampleSet, SampleSetPlot
from typing import Callable, Generator

LHS = LHSampler(d = 2)
err = 1e-9

def expsave(id:str, data, mode:str):
    with open("./data/" + id + ".json", "w+") as f:
        if(mode == "w"):
            json.dump(data, f)
        elif(mode == "a"):
            prev = expget(id)
            json.dump(prev.append(data), f)

def expget(id:str):
    with open("./data/" + id + ".json") as f:
        return json.load(f)


# P-dimensional Reimann integration 

In [194]:
class midpointIntegration:
    def __init__(self, d:int):
        if d <= 0:
            raise ValueError("Dimension must be positive")
        self.d = d
    
    def __gen_indexes(self, Ns, borders = False) -> Generator[list, None, None]:
        index = [0] * self.d
        if not borders:
            Ns = (np.array(Ns) - 1).tolist()
        yield index
        for _ in range(np.prod(Ns)):
            for p in reversed(range(self.d)):
                if index[p] + 1 == Ns[p]:
                    if p == 0:
                        return
                    index[p] = 0
                else:
                    index[p] += 1
                    break
            yield index

    def __relindex(self, abs_index: int, Ns: list[int]):
        rel = np.zeros((self.d), dtype=int)
        r = abs_index
        for p in range(0, self.d):
            rel[p] = floor(r/Ns[p])
            r = r % Ns[p]
        rel[-1] = r
        return rel

    def __absindex(self, rel_index:list[int], Ns:list[int]):
        abs = 0
        for p in range(0, self.d - 1):
            abs += Ns[p] * rel_index[p]
        abs += rel_index[-1]
        return abs

    def __shift_on_axis(self, index, axis: int, Ns: list[int], increment: int = 1):
        if type(index) is int:
            index = self.__relindex(index, Ns)
        rtr = [*index]
        rtr[axis] += increment
        if type(index) is int:
            return self.__absindex(rtr)
        else:
            return rtr

    def midpoint(self, xs, index: list[int], Ns: list[int]):
        m = np.zeros((self.d))
        for p in range(self.d):
            shifted_index = self.__shift_on_axis(index, p, Ns)
            m[p] = (xs[*index, p] + xs[*shifted_index, p])/2
        return m

    # lbs = lower_boundaries
    # ubs = upper_boundaries
    def integrate(self, f:Callable, Ns:list[int], lbs: list[int], ubs: list[int]):
        if len(Ns) != self.d:
            raise ValueError("Ns must be a list of integers ", self.d, " long")
        
        xs = np.zeros((*Ns, self.d))
        for i in self.__gen_indexes(Ns, borders = True):
            xs[*i] = [((i[k]/Ns[k])*(ubs[k] - lbs[k]))+lbs[k] for k in range(self.d)]
         
        I = 0
        for i in self.__gen_indexes(Ns, borders = False):
            I += f(self.midpoint(xs, i, Ns))
        I *= 1/np.prod(Ns)
        return I

def fun3(x):
    if type(x) is int:
        x = [x]
    return 3*np.sin(x[0])*(np.cos(x[1])**3)

d = 2
calc = midpointIntegration(d)
lbs, ubs = [0] * d, [1] * d 
Ns = [2000] * d
calc.integrate(fun3, Ns, lbs, ubs)


0.8856481815092144

In [169]:
pippo = np.zeros((10,10,10,5))
idx = np.array([1,2,3]).tolist()
type(idx)
# pippo[*idx, 3]


list

In [135]:
def graph(f, n = 1000, lim = [-1, 1], xs = None):
    fig, ax = plt.subplots(1,1)
    ax.set_xlim(lim[0])
    ax.set_ylim(lim[1])
    if xs is None:
        xs = np.linspace(lim[0], lim[1], n)
    ax.plot(xs, f(xs))



## SOBOL

In [97]:
# Define the complex function
def fun1(x):
    if type(x) is not list:
        x = [x]
    return np.exp(-np.sum((x - 1/np.arange(1, len(x)+1))**2)) * np.cos(np.sum(x))

def fun2(x):
    return -np.sin(x) + np.exp(- (x**2)) * np.cos(x)

# Define the integration domain and dimension
d = 1
domain = np.array([[-1, 1]] * d)

# Sampling methods (example with Sobol sequence)
sampler = qmc.Sobol(d)
samples = sampler.random_base2(m=15)  # 2^m samples
samples = qmc.scale(samples, domain[:, 0], domain[:, 1])  # Scale to domain

# Evaluate the function at sample points
values = np.apply_along_axis(fun2, 1, samples)

# Estimate the integral
integral_estimate = np.mean(values) * np.prod(domain[:, 1] - domain[:, 0])


print(f"Estimated Integral: {integral_estimate}")
# Repeat for different sampling methods and collect results


Estimated Integral: 1.3123487270237888


In [ ]:

lim, n = [-3, 3],1000
fig, ax = plt.subplots(1,1)
ax.set_xlim(lim)
xs = np.linspace(lim[0], lim[1], n)
ys = [fun1(x) for x in xs]
ax.plot(xs, ys)